## 対象のチャンネルから全video_idを取得する

In [ ]:
#検索したいチャンネルのchannelId
# CHANNEL_ID = 'UCi1s8Tz0Xmi_wW1vdJlx3xQ'
# OUTPUT_FILE = 'ranking_rabo'

# ランキングラボ
# CHANNEL_ID = 'UCi1s8Tz0Xmi_wW1vdJlx3xQ'
# OUTPUT_FILE = 'ranking_rabo'

# オールマイティラボ
CHANNEL_ID = 'UC34oyKa3OSUt8IXfi9MtrEA'
OUTPUT_FILE = 'allmity_labo'

In [ ]:
import requests
import json
import pandas as pd

with open('/Users/kii/work/access_analysis/json/youtube_data.json') as f:
    data = json.load(f)
API_KEY = data['API_KEY']

#SearchリソースのリクエストURL指定
url = 'https://www.googleapis.com/youtube/v3/'

'''
対象チャンネルの全ての動画IDを取得
'''
resource = 'search'
request_url = url + resource

#空のリストを準備
videoIdList = []

#pageTokenの初期化
pageToken = None

while True:
    #パラメータ設定
    params = {
        'key': API_KEY, #1.で取得したAPIキーを入力
        'part': 'id', #videoIdを含むプロパティ

        'channelId': CHANNEL_ID, #検索したいチャンネルのchannelId
        
        'order': 'date', #検索結果を時系列順に表示
        'type': 'video', #動画のみを検索（プレイリストなどを検索結果に含まない）

        'maxResults': 50, #1リクエストあたりの検索結果の動画数（デフォルトは5、最大で50まで設定可能）
        'pageToken': pageToken #リクエストのトークン
    }
    
    #リクエスト実行
    search_result = requests.get(request_url, params=params)
    
    #json形式の結果を格納
    search_result_json = search_result.json()
    
    #videoIdの抽出
    for item in search_result_json['items']:
        videoIdList.append(item['id']['videoId'])
    
    #検索結果を全て表示できていない場合はpageTokenを更新
    if 'nextPageToken' in search_result_json:
        pageToken = search_result_json['nextPageToken']
    else:
        break
        
#データ取得した動画数の確認
print(len(videoIdList))

'''
全ての動画の詳細情報を取得
'''
#VideosリソースのリクエストURL指定
resource = 'videos'
request_url = url + resource

#データを追加する空のリストを準備
data = []

#videoIdListの分割
videoIdList_splitted = []
for i in range(0, len(videoIdList), 50):
    videoIdList_splitted.append(videoIdList[i:i+50])

#各グループごとにリクエスト実行
for videoId_grp in videoIdList_splitted:
    #パラメータ設定
    params = {
        'key': API_KEY,
        'part': 'snippet,statistics',
        'id': ','.join(videoId_grp)
    }
    
    #リクエスト実行
    videos_result = requests.get(request_url, params=params)
    videos_result_json = videos_result.json()

    #動画情報の抽出
    for item in videos_result_json['items']:
        channelTitle = item['snippet']['channelTitle']
        videoId = item['id']
        title = item['snippet']['title']
        publishedAt = item['snippet']['publishedAt']
        description = item['snippet']['description']
        tags = item['snippet'].get('tags', 0)
        commentCount = item['statistics'].get('commentCount', 0)
        likeCount = item['statistics'].get('likeCount', 0)
        viewCount = item['statistics'].get('viewCount', 0)

        #結果を逐次的に追加
        data.append([channelTitle, videoId, title, publishedAt, description, tags, commentCount, likeCount, viewCount])

#DataFrameの形で出力
columns = ['channelTitle', 'videoId', 'title', 'publishedAt', 'description', 'tags', 'commentCount', 'likeCount', 'viewCount']
df = pd.DataFrame(data, columns=columns)

df['viewCount'] = df['viewCount'].astype(int)
df = df.sort_values(by='viewCount', ascending=False)

output_path = f'./output/{OUTPUT_FILE}.xlsx'
df.to_excel(output_path, index=False)